# Generative Pre-Training from Molecules

In this notebook, we demonstrate how to pretrain
[HuggingFace](https://huggingface.co/transformers/)
[GPT-2](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel) language model 
on a SMILES corpus. [SMILES](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html) is
a language construct for representing molecules, with its unique syntax and vocabulary of 
molecular constituents. Pretraining GPT-2 on large and diverse corpora allows capturing
general representations of molecules capable of being transferred to such downstream tasks as
molecular-property prediction and low-data de novo molecular design.

---

*Author: Sanjar Adilov*

*Paper: [Generative Pre-Training from Molecules](https://doi.org/10.33774/chemrxiv-2021-5fwjd)*,
*DOI: 10.33774/chemrxiv-2021-5fwjd*

*Package: https://github.com/sanjaradylov/smiles-gpt*

## Main Package

Our [`smiles_gpt`](https://github.com/sanjaradylov/smiles-gpt/tree/master/smiles_gpt)
package implements
[pytorch-lightning](https://www.pytorchlightning.ai/)-compatible modules for data loading,
model training and testing. The SMILES tokenizer and downstream regression and
single-/multi-output classification models are also compatible with HuggingFace API.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
import sys
sys.path.append('/home/jmwang/drugai/iupac-gpt')

try:
    import iupac_gpt as gpt
except ImportError:
    import sys
    sys.path.extend([".."])  # Parent directory stores `smiles_gpt` package.
    import smiles_gpt as gpt

For demonstration purposes, we use only 10K subset of PubChem data made available by
[ChemBERTa](https://arxiv.org/abs/2010.09885) developers. The original model was pretrained
on the first 5M compounds with the following hyperparameters:
```python
hyperparams = {"batch_size": 128, "max_epochs": 2, "max_length": 512,
               "learning_rate": 5e-4, "weight_decay": 0.0,
               "adam_eps": 1e-8, "adam_betas": (0.9, 0.999),
               "scheduler_T_max": 150_000, "final_learning_rate": 5e-8,
               "vocab_size": 1_000, "min_frequency": 2, "top_p": 0.96,
               "n_layer": 4, "n_head": 8, "n_embd": 512}
```

In [9]:
# 10K subset of PubChem SMILES dataset.
filename = "../data/iupacs_logp.csv"
# Directory to serialize a tokenizer and model.
checkpoint = "../checkpoints/iupac"
tokenizer_filename = f"{checkpoint}/tokenizer.json"

# Tokenizer, model, optimizer, scheduler, and trainer hyperparameters.
hyperparams = {"batch_size": 256, "max_epochs": 10, "max_length": 256,
               "learning_rate": 5e-4, "weight_decay": 0.0,
               "adam_eps": 1e-8, "adam_betas": (0.9, 0.999),
               "scheduler_T_max": 1_000, "final_learning_rate": 5e-8,
               "vocab_size": 200, "min_frequency": 2, "top_p": 0.96,
               "n_layer": 8, "n_head": 8, "n_embd": 256}

gpus = 1  # Specify either a list of GPU devices or an integer (0 for no GPU).
num_workers = 4  # Number of dataloader worker processes.

## Tokenization

`smiles_gpt.SMILESBPETokenizer` first splits SMILES strings into characters, runs
byte-pair encoding, and augments the resulting list with `"<s>"` (beginning-of-SMILES) and
`"</s>"` (end-of-SMILES) special tokens. `smiles_gpt.SMILESAlphabet` stores 72 possible
characters as an initial vocabulary.

In [12]:
alphabet = list(gpt.SMILESAlphabet().get_alphabet())
tokenizer = gpt.SMILESBPETokenizer(dropout=None)
tokenizer.train(filename,
                vocab_size=hyperparams["vocab_size"] + len(alphabet),
                min_frequency=hyperparams["min_frequency"],
                initial_alphabet=alphabet)
tokenizer.save_model(checkpoint)
tokenizer.save(tokenizer_filename)
tokenizer

Tokenizer(vocabulary_size=272, model=BPE, unk_token=<unk>, suffix=, dropout=None)

[`SMILESBPETokenizer`](https://github.com/sanjaradylov/smiles-gpt/blob/master/smiles_gpt/tokenization.py#L23)
inherits `BaseTokenizer` from
[Tokenizers](https://huggingface.co/docs/tokenizers/python/latest/index.html). It is already
useful by itself, however, to make it more convenient and follow HuggingFace API, we load
`transformers.PreTrainedTokenizerFast` instance of our tokenizer:

In [11]:
from pprint import pprint

tokenizer = gpt.SMILESBPETokenizer.get_hf_tokenizer(
    tokenizer_filename, model_max_length=hyperparams["max_length"])

smiles_string = "CC(Cl)=CCCC=C(C)Cl"
smiles_encoded = tokenizer(smiles_string)
smiles_merges = tokenizer.convert_ids_to_tokens(smiles_encoded["input_ids"])

pprint(smiles_encoded)
pprint(smiles_merges)

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [1, 94, 104, 262, 27, 94, 94, 27, 31, 178, 31, 69, 2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
['<s>', 'CC', '(C', 'l)', '=', 'CC', 'CC', '=', 'C', '(C)', 'C', 'l', '</s>']


## Data Module

[`smiles_gpt.LMDataModule`](https://github.com/sanjaradylov/smiles-gpt/blob/master/smiles_gpt/data.py#L248)
is a lightning data module that loads SMILES data, encodes them
with `tokenizer`, and returns pytorch data loader with
`transformers.DataCollatorForLanguageModeling` collator. Encodings contain tensors of shape
`hyperparameters["max_length"]`: `"input_ids"` and `"lables"`.

In [13]:
datamodule = gpt.LMDataModule(filename, tokenizer,
                              batch_size=hyperparams["batch_size"],
                              num_workers=num_workers)
datamodule.setup()

batch = next(iter(datamodule.train_dataloader()))
pprint(batch)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jmwang/anaconda3/envs/smiles-gpt/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/jmwang/anaconda3/envs/smiles-gpt/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/jmwang/anaconda3/envs/smiles-gpt/lib/python3.8/site-packages/transformers/data/data_collator.py", line 341, in __call__
    batch = {"input_ids": _collate_batch(examples, self.tokenizer)}
  File "/home/jmwang/anaconda3/envs/smiles-gpt/lib/python3.8/site-packages/transformers/data/data_collator.py", line 208, in _collate_batch
    if tokenizer._pad_token is None:
AttributeError: 'SMILESBPETokenizer' object has no attribute '_pad_token'


## GPT-2 Model

Now we load HuggingFace
[`GPT2LMHeadModel`](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel)
with the configuration composed of previously
defined model hyperparameters. The model processes mini-batch of input ids and labels, then
returns predictions and cross-entropy loss between labels and predictions.

In [7]:
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=tokenizer.vocab_size,
                    bos_token_id=tokenizer.bos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    n_layer=hyperparams["n_layer"],
                    n_head=hyperparams["n_head"],
                    n_embd=hyperparams["n_embd"],
                    n_positions=hyperparams["max_length"],
                    n_ctx=hyperparams["max_length"])
model = GPT2LMHeadModel(config)

outputs = model(**batch)
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values'])

## Trainer

GPT-2 is trained with autoregressive language modeling objective:
$$
P(\boldsymbol{s}) = P(s_1) \cdot P(s_2 | s_1) \cdots P(s_T | s_1, \ldots, s_{T-1}) =
\prod_{t=1}^{T} P(s_t | s_{j < t}),
$$
where $\boldsymbol{s}$ is a tokenized (encoded) SMILES string, $s_t$ is a token from pretrained 
vocabulary $\mathcal{V}$.

We use `pytorch_lightning.Trainer` to train GPT-2. Since `Trainer` requires lightning modules,
we import our
[`smiles_gpt.GPT2LitModel`](https://github.com/sanjaradylov/smiles-gpt/blob/master/smiles_gpt/language_modeling.py#L10)
wrapper that implements training phases for
`GPT2LMHeadModel`, configures an `Adam` optimizer with `CosineAnnealingLR` scheduler, and
logs average perplexity every epoch.

In [8]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

trainer = Trainer(
    gpus=gpus,
    max_epochs=hyperparams["max_epochs"],
    callbacks=[EarlyStopping("ppl", 0.2, 2)],
    auto_lr_find=False,  # Set to True to search for optimal learning rate.
    auto_scale_batch_size=False  # Set to True to scale batch size
    # accelerator="dp"  # Uncomment for GPU training.
)
lit_model = gpt.GPT2LitModel(
    model,
    batch_size=hyperparams["batch_size"],
    learning_rate=hyperparams["learning_rate"],
    final_learning_rate=hyperparams["final_learning_rate"],
    weight_decay=hyperparams["weight_decay"],
    adam_eps=hyperparams["adam_eps"],
    adam_betas=hyperparams["adam_betas"],
    scheduler_T_max=hyperparams["scheduler_T_max"],
    save_model_every=10, checkpoint=checkpoint)
trainer.fit(lit_model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/jmwang/anaconda3/envs/smiles-gpt/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name        | Type            | Params
------------------------------------------------
0 | transformer | GPT2LMHeadModel | 6.5 M 
------------------------------------------------
6.5 M     Trainable params
0         Non-trainable params
6.5 M     Total params
25.815    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the enviro

## Interpretability

[BertViz](https://github.com/jessevig/bertviz) inspects attention heads of transformers
capturing specific patterns in data. Each head can be representative of some syntactic
or short-/long-term relationships between tokens.

In [9]:
import torch
from bertviz import head_view

smiles = "CC[NH+](CC)C1CCC([NH2+]C2CC2)(C(=O)[O-])C1"
inputs = tokenizer(smiles, add_special_tokens=False, return_tensors="pt")
input_ids_list = inputs["input_ids"].tolist()[0]
model = GPT2LMHeadModel.from_pretrained(checkpoint, output_attentions=True)
attention = model(torch.LongTensor(input_ids_list))[-1]
tokens = tokenizer.convert_ids_to_tokens(input_ids_list)

# Don't worry if a snippet is not displayed---just rerun this cell.
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [10]:
from bertviz import model_view

# Don't worry if a snippet is not displayed---just rerun this cell.
model_view(attention, tokens)

<IPython.core.display.Javascript object>

## Sampling

Finally, we generate novel SMILES strings with top-$p$ sampling$-$i.e., sampling from the
smallest vocabulary subset $\mathcal{V}^{(p)} \subset \mathcal{V}$ s.t. it takes up the most
probable tokens whose cumulative probability mass exceeds $p$, $0 < p < 1$. Model
terminates the procedure upon encountering `"</s>"` or reaching maximum number
`hyperparams["max_length"]`. Special tokens are eventually removed.

In [11]:
import tqdm

model.eval()  # Set the base model to evaluation mode.

generated_smiles_list = []
n_generated = 10000

for _ in tqdm.tqdm(range(n_generated)):
    # Generate from "<s>" so that the next token is arbitrary.
    smiles_start = torch.LongTensor([[tokenizer.bos_token_id]])
    # Get generated token IDs.
    generated_ids = model.generate(smiles_start,
                                   max_length=hyperparams["max_length"],
                                   do_sample=True,top_p=hyperparams["top_p"],
                                   repetition_penalty=1.2,
                                   pad_token_id=tokenizer.eos_token_id)
    # Decode the IDs into tokens and remove "<s>" and "</s>".
    generated_smiles = tokenizer.decode(generated_ids[0],
                                        skip_special_tokens=True)
    generated_smiles_list.append(generated_smiles)

generated_smiles_list[:10]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [18:50:57<00:00,  6.79s/it]


['CCCC(C)(O)C1CCC2C3C(O)CC4=CC(=O)CCC4(C)C3C(c3ccc4c(c3)OCO4)CC21C',
 'Cc1ccc(-n2nc(C)n3c(cc4sc5ccccc5c43)c2=O)cc1F',
 'Cc1nc(NC(C)c2ccc(-c3ccc(CNC4CC4)cc3)cc2)c2cc(C3CCC(C(=O)N(C)C)CC3)ccc2n1',
 'Cn1cccc1C(=O)Nc1ccc2c(c1)CCCN2S(C)(=O)=O',
 'Cc1ccnc2c1C(=NN=C(N)N)CC(c1cc(Br)ccc1Cl)C2',
 'COC(=O)c1cc2c(cc1NC(=O)COC(=O)C(Cc1c(C)[nH]c3ccccc13)NC(C)=O)OCO2',
 'O=C(CNC(C(F)(F)F)(c1ccccc1)c1ccccc1)NC(c1ccc(Cl)cc1)c1ccc(Cl)cc1',
 'CCOc1nc(N)nc2c1c(C)nn2-c1ccc(C(C)C)cc1Cl',
 'CC(C)Oc1ccc(S(=O)(=O)Nc2ccccc2C(=O)N2CCOCC2)cc1',
 'COc1cc(S(=O)(=O)N2CCCCC2)ccc1NC(=O)CNCC1COc2ccccc2O1']

In [ ]:
import tqdm

model.eval()  # Set the base model to evaluation mode.

generated_smiles_list = []
n_generated = 10000

for _ in tqdm.tqdm(range(n_generated)):
    # Generate from "<s>" so that the next token is arbitrary.
    smiles_start = torch.LongTensor([[tokenizer.bos_token_id]])
    # Get generated token IDs.
    generated_ids = model.generate(smiles_start,
                                   max_length=hyperparams["max_length"],
                                   do_sample=True,top_p=hyperparams["top_p"],
                                   repetition_penalty=2.0,
                                   pad_token_id=tokenizer.eos_token_id)
    # Decode the IDs into tokens and remove "<s>" and "</s>".
    generated_smiles = tokenizer.decode(generated_ids[0],
                                        skip_special_tokens=True)
    generated_smiles_list.append(generated_smiles)

generated_smiles_list[:10]

 23%|████████████████████████                                                                                | 2311/10000 [4:24:01<14:23:34,  6.74s/it]

In [ ]:
import numpy as np
import pandas as pd

df2 = pd.DataFrame(generated_smiles_list, columns=['smiles']) 

df2.to_csv("smi3GPT2-gen30K.csv",index=None,mode='a')

In [ ]:
from rdkit.Chem import MolFromSmiles
from rdkit.RDLogger import DisableLog
from rdkit.Chem.Draw import MolsToGridImage
DisableLog("rdApp.*")

valid_molecules = []
for smiles in generated_smiles_list:
    molecule = MolFromSmiles(smiles)
    if molecule is not None:
        valid_molecules.append(molecule)

MolsToGridImage(valid_molecules[:30])

## Further Reading

The pretrained model can be used for transferring knowledge to downstream tasks
including molecular property prediction. Check out
[`smiles_gpt`](https://github.com/sanjaradylov/smiles-gpt/tree/master/smiles_gpt)
repository for implementation details and
[smiles-gpt/scripts](https://github.com/sanjaradylov/smiles-gpt/scripts)
directory for single-/multi-output classification scripts. To evaluate generated
molecules, consider distribution-learning metrics from
[moleculegen-ml](https://github.com/sanjaradylov/moleculegen-ml).

If you find `smiles_gpt` as well as examples from this repository useful in your
research, please consider citing
> Adilov, Sanjar (2021): Generative Pre-Training from Molecules. ChemRxiv. Preprint. https://doi.org/10.33774/chemrxiv-2021-5fwjd